In [ ]:
# Bronze step crime
import pandas as pd

# Load CSV file
csv_file_path = 'c:/Users/darka/Desktop/Projets/Elexxion/crime/crime_2016_2024_departement.csv'

# Transform CSV in dataframe
df = pd.read_csv(csv_file_path, sep=';', encoding='utf-8')

# Save as Parquet with '_bronze' prefix
parquet_file_path = 'c:/Users/darka/Desktop/Projets/Elexxion/crime/crime_2016_2024_departement_bronze.parquet'
df.to_parquet(parquet_file_path, index=False)

print(f'File saved as Parquet: {parquet_file_path}')

In [ ]:
# Silver step crime
import pandas as pd

# Load the bronze Parquet file
bronze_file_path = 'c:/Users/darka/Desktop/Projets/Elexxion/crime/crime_2016_2024_departement_bronze.parquet'
df = pd.read_parquet(bronze_file_path)

# Standardize column names
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Remove duplicates
df = df.drop_duplicates()

# Remove empty rows
df = df.dropna()

# Colums renaming
df.rename(columns={'code_departement': 'departement'}, inplace=True)
df.rename(columns={'code_region': 'region'}, inplace=True)
df.rename(columns={'insee_pop': 'population'}, inplace=True)
df.rename(columns={'indicateur': 'type'}, inplace=True)

# Colums deleting
df = df.drop(columns=["unite_de_compte", "insee_log", "insee_pop_millesime", "insee_log_millesime"])

# Save as silver Parquet file
silver_file_path = 'c:/Users/darka/Desktop/Projets/Elexxion/crime/crime_2016_2024_departement_silver.parquet'
df.to_parquet(silver_file_path, index=False)


In [ ]:
# Gold step crime
import pandas as pd

# Load the silver Parquet file
silver_file_path = 'c:/Users/darka/Desktop/Projets/Elexxion/crime/crime_2016_2024_departement_silver.parquet'
df = pd.read_parquet(silver_file_path)

# Aggregate data: sum 'nombre' per department and year
gold_df = df.groupby(['departement', 'annee', 'region', 'population'])['nombre'].sum().reset_index()

# Save as gold Parquet file
gold_file_path = 'c:/Users/darka/Desktop/Projets/Elexxion/crime/crime_2016_2024_departement_gold.parquet'
gold_df.to_parquet(gold_file_path, index=False)